In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from torch.utils.data import DataLoader

In [2]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder(
                    root='./data/trainset/',
                    transform=transforms
                    )

test_data = torchvision.datasets.ImageFolder(
                    root= './data/test/', 
                    transform=transforms)

train_set_ratio = 0.8
size = [int(len(train_data)*train_set_ratio), len(train_data) - int(len(train_data)*train_set_ratio)]

train_dataset, valid_dataset = torch.utils.data.random_split(dataset=train_data, lengths=size)

train_loader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

valid_loader = torch.utils.data.DataLoader(
                    valid_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )



test_loader = torch.utils.data.DataLoader(
                    test_data,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )
print(train_data)
print(test_data)

Dataset ImageFolder
    Number of datapoints: 19998
    Root Location: ./data/trainset/
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None
Dataset ImageFolder
    Number of datapoints: 4999
    Root Location: ./data/test/
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None


In [18]:
class Classifier(nn.Module):
    """Convnet Classifier"""
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 5
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 6
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(256, 2)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())

In [19]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


## Hyper-parameters of the model

In [40]:
learning_rate = 1e-3

In [41]:
clf = Classifier()
if cuda_available:
    clf = clf.cuda()
optimizer = torch.optim.Adam(clf.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [42]:
epoches= 50
for epoch in range(epoches):
    losses = []
    # Train
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()
        outputs = clf(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())
        
        if batch_idx%50==0:
            print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))
    
    # Evaluate
    clf.eval()
    total = 0
    correct = 0
    for batch_idx, (inputs, targets) in enumerate(valid_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = clf(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print('--------------------------------------------------------------')
    clf.train()

Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
    w.join()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
A

Epoch : 0 Loss : 0.699 
Epoch : 0 Loss : 0.694 
Epoch : 0 Loss : 0.694 
Epoch : 0 Loss : 0.694 
Epoch : 0 Loss : 0.693 


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
    w.join()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/proce

Epoch : 0 Test Acc : 55.000
--------------------------------------------------------------


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
    w.join()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/proce

...

Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py",

Epoch : 2 Test Acc : 73.000
--------------------------------------------------------------


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f7cb90a9048>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
    w.join()
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/home/mehrzaed/Software/anaconda3/lib/python3.7/multiprocessing/proce

Epoch : 3 Loss : 0.512 
Epoch : 3 Loss : 0.479 
Epoch : 3 Loss : 0.484 
Epoch : 3 Loss : 0.486 
Epoch : 3 Loss : 0.488 
Epoch : 3 Test Acc : 76.000
--------------------------------------------------------------
Epoch : 4 Loss : 0.447 
Epoch : 4 Loss : 0.446 
Epoch : 4 Loss : 0.444 
Epoch : 4 Loss : 0.437 
Epoch : 4 Loss : 0.431 
Epoch : 4 Test Acc : 80.000
--------------------------------------------------------------
Epoch : 5 Loss : 0.288 
Epoch : 5 Loss : 0.371 
Epoch : 5 Loss : 0.374 
Epoch : 5 Loss : 0.384 
Epoch : 5 Loss : 0.383 
Epoch : 5 Test Acc : 80.000
--------------------------------------------------------------
Epoch : 6 Loss : 0.337 
Epoch : 6 Loss : 0.315 
Epoch : 6 Loss : 0.323 
Epoch : 6 Loss : 0.326 
Epoch : 6 Loss : 0.331 
Epoch : 6 Test Acc : 80.000
--------------------------------------------------------------
Epoch : 7 Loss : 0.368 
Epoch : 7 Loss : 0.287 
Epoch : 7 Loss : 0.288 
Epoch : 7 Loss : 0.294 
Epoch : 7 Loss : 0.296 
Epoch : 7 Test Acc : 82.000
--------

Epoch : 41 Loss : 0.003 
Epoch : 41 Loss : 0.009 
Epoch : 41 Loss : 0.007 
Epoch : 41 Loss : 0.014 
Epoch : 41 Loss : 0.021 
Epoch : 41 Test Acc : 81.000
--------------------------------------------------------------
Epoch : 42 Loss : 0.016 
Epoch : 42 Loss : 0.016 
Epoch : 42 Loss : 0.018 
Epoch : 42 Loss : 0.023 
Epoch : 42 Loss : 0.025 
Epoch : 42 Test Acc : 81.000
--------------------------------------------------------------
Epoch : 43 Loss : 0.005 
Epoch : 43 Loss : 0.012 
Epoch : 43 Loss : 0.022 
Epoch : 43 Loss : 0.025 
Epoch : 43 Loss : 0.022 
Epoch : 43 Test Acc : 80.000
--------------------------------------------------------------
Epoch : 44 Loss : 0.002 
Epoch : 44 Loss : 0.016 
Epoch : 44 Loss : 0.021 
Epoch : 44 Loss : 0.020 
Epoch : 44 Loss : 0.023 
Epoch : 44 Test Acc : 82.000
--------------------------------------------------------------
Epoch : 45 Loss : 0.012 
Epoch : 45 Loss : 0.013 
Epoch : 45 Loss : 0.014 
Epoch : 45 Loss : 0.014 
Epoch : 45 Loss : 0.014 
Epoch :

In [44]:
results = []
for batch_idx, (inputs, targets) in enumerate(valid_loader):
    if cuda_available:
        inputs, targets = inputs.cuda(), targets.cuda()
    outputs = clf(inputs)
    _, predicted = torch.max(outputs.data, 1)
    results.append(predicted)
    


In [49]:
df = pd.DataFrame.from_records(np.array(results))
print(df)

                            0                           1   \
0   tensor(1, device='cuda:0')  tensor(0, device='cuda:0')   
1   tensor(1, device='cuda:0')  tensor(1, device='cuda:0')   
2   tensor(1, device='cuda:0')  tensor(0, device='cuda:0')   
3   tensor(0, device='cuda:0')  tensor(0, device='cuda:0')   
4   tensor(1, device='cuda:0')  tensor(1, device='cuda:0')   
5   tensor(1, device='cuda:0')  tensor(1, device='cuda:0')   
6   tensor(1, device='cuda:0')  tensor(0, device='cuda:0')   
7   tensor(0, device='cuda:0')  tensor(0, device='cuda:0')   
8   tensor(1, device='cuda:0')  tensor(0, device='cuda:0')   
9   tensor(0, device='cuda:0')  tensor(1, device='cuda:0')   
10  tensor(0, device='cuda:0')  tensor(0, device='cuda:0')   
11  tensor(1, device='cuda:0')  tensor(0, device='cuda:0')   
12  tensor(1, device='cuda:0')  tensor(1, device='cuda:0')   
13  tensor(1, device='cuda:0')  tensor(1, device='cuda:0')   
14  tensor(0, device='cuda:0')  tensor(0, device='cuda:0')   
15  tens